# Real SAR Urban Change Detection with Sentinel-1 Data

This notebook demonstrates urban change detection using real Sentinel-1 SAR data.
We will download, preprocess, and analyze time-series SAR patches to detect changes.

## Requirements:
- Copernicus Open Access Hub account (https://scihub.copernicus.eu/dhus/)
- Set environment variables: COPERNICUS_USER and COPERNICUS_PASSWORD
- Internet connection for data download

In [ ]:
# Install required packages
import sys
import subprocess

def install_package(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

# Install SAR4CET and dependencies
try:
    import sar4cet
except ImportError:
    print('Installing SAR4CET...')
    install_package('sentinelsat')
    install_package('rasterio')
    install_package('geopandas')
    install_package('matplotlib')
    install_package('numpy')
    install_package('scipy')
    install_package('scikit-image')
    install_package('opencv-python')
    
    # Install SAR4CET package
    import os
    
    # Method 1: Try installing from GitHub
    try:
        print('Installing SAR4CET from GitHub...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'git+https://github.com/naik15/SAR4CET.git'])
        print('Successfully installed from GitHub!')
    except subprocess.CalledProcessError:
        print('GitHub install failed, trying alternative methods...')
        
        # Method 2: Try local installation if we're in a local environment
        try:
            if os.path.exists('../setup.py'):
                print('Found local setup.py, installing locally...')
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-e', '..'])
                print('Successfully installed locally!')
            else:
                raise FileNotFoundError('No local setup.py found')
        except (subprocess.CalledProcessError, FileNotFoundError):
            # Method 3: Clone and install
            print('Cloning repository and installing...')
            if not os.path.exists('SAR4CET'):
                subprocess.check_call(['git', 'clone', 'https://github.com/naik15/SAR4CET.git'])
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-e', './SAR4CET'])
            print('Successfully installed from cloned repository!')

print('All packages installed successfully!')

# Verify SAR4CET installation
try:
    import sar4cet
    print(f'SAR4CET version: {getattr(sar4cet, "__version__", "unknown")}')
    print('SAR4CET successfully imported!')
except ImportError as e:
    print(f'Warning: SAR4CET import failed: {e}')
    print('Please restart the runtime and try again.')

In [ ]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import rasterio
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import box
import warnings
warnings.filterwarnings('ignore')

# Import SAR4CET modules
from sar4cet import preprocessing, change_detection, visualization, utils

print('Libraries imported successfully!')

## 1. Set up Copernicus Hub Credentials

You need to register at https://scihub.copernicus.eu/dhus/ and set your credentials.

In [ ]:
# Set up Copernicus credentials
# Option 1: Set environment variables (recommended)
# export COPERNICUS_USER="your_username"
# export COPERNICUS_PASSWORD="your_password"

# Option 2: Set them here (not recommended for security)
# os.environ['COPERNICUS_USER'] = 'your_username'
# os.environ['COPERNICUS_PASSWORD'] = 'your_password'

# Check if credentials are set
if 'COPERNICUS_USER' in os.environ and 'COPERNICUS_PASSWORD' in os.environ:
    print('Credentials found!')
    print(f"Username: {os.environ['COPERNICUS_USER']}")
else:
    print('WARNING: Copernicus credentials not found!')
    print('Please set COPERNICUS_USER and COPERNICUS_PASSWORD environment variables')

## 2. Define Area of Interest and Time Period

We'll focus on a small urban area to keep download sizes manageable.

In [ ]:
# Define area of interest (small patch for demonstration)
# Example: San Francisco Bay Area (small subset)
aoi_bbox = [-122.45, 37.75, -122.40, 37.80]  # [lon_min, lat_min, lon_max, lat_max]

# Alternative areas (uncomment one):
# London, UK (small patch)
# aoi_bbox = [-0.15, 51.50, -0.10, 51.52]

# Berlin, Germany (small patch)
# aoi_bbox = [13.35, 52.50, 13.40, 52.52]

# Time period (keep short to limit data volume)
start_date = '2023-01-01'
end_date = '2023-06-30'

print(f"Area of Interest: {aoi_bbox}")
print(f"Time Period: {start_date} to {end_date}")

# Create AOI geometry for visualization
aoi_geom = box(aoi_bbox[0], aoi_bbox[1], aoi_bbox[2], aoi_bbox[3])
aoi_gdf = gpd.GeoDataFrame([1], geometry=[aoi_geom], crs='EPSG:4326')

# Plot AOI
fig, ax = plt.subplots(figsize=(8, 6))
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
ax.set_title('Area of Interest')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Search and Download Sentinel-1 Data

In [ ]:
# Create download directory
download_dir = 'sentinel1_data'
os.makedirs(download_dir, exist_ok=True)

try:
    # Search for Sentinel-1 data
    print('Searching for Sentinel-1 data...')
    search_results = preprocessing.search_sentinel1(
        aoi=aoi_bbox,
        start_date=start_date,
        end_date=end_date,
        polarization='VV VH',
        product_type='GRD'
    )
    
    print(f"Found {len(search_results)} Sentinel-1 products")
    
    # Display search results
    if len(search_results) > 0:
        print('\nFirst 5 products:')
        for i, (product_id, product_info) in enumerate(list(search_results.items())[:5]):
            print(f"{i+1}. {product_info['title']} - {product_info['beginposition']}")
    
    # Limit downloads to first 3-5 products to keep data manageable
    max_downloads = min(5, len(search_results))
    limited_results = dict(list(search_results.items())[:max_downloads])
    
    print(f"\nDownloading {max_downloads} products...")
    downloaded_files = preprocessing.download_sentinel1(
        aoi=aoi_bbox,
        start_date=start_date,
        end_date=end_date,
        download_dir=download_dir,
        max_products=max_downloads
    )
    
    print(f"Downloaded {len(downloaded_files)} files")
    for file in downloaded_files:
        print(f"  - {os.path.basename(file)}")
        
except Exception as e:
    print(f"Error during download: {e}")
    print("\nFalling back to simulated data for demonstration...")
    
    # Create simulated data as fallback
    downloaded_files = []
    for i in range(5):
        # Create simulated SAR data
        np.random.seed(42 + i)
        base_image = np.random.gamma(2, 0.5, (512, 512))
        
        # Add some urban-like structures
        base_image[100:150, 100:150] *= 1.5  # Building block
        base_image[200:250, 200:300] *= 1.3  # Another area
        
        # Add temporal changes
        if i > 2:
            base_image[300:350, 300:400] *= 2.0  # New development
        
        # Save as GeoTIFF
        filename = f"{download_dir}/simulated_sar_{i+1}.tif"
        
        # Create rasterio profile
        profile = {
            'driver': 'GTiff',
            'height': base_image.shape[0],
            'width': base_image.shape[1],
            'count': 1,
            'dtype': base_image.dtype,
            'crs': 'EPSG:4326',
            'transform': rasterio.transform.from_bounds(
                aoi_bbox[0], aoi_bbox[1], aoi_bbox[2], aoi_bbox[3],
                base_image.shape[1], base_image.shape[0]
            )
        }
        
        with rasterio.open(filename, 'w', **profile) as dst:
            dst.write(base_image, 1)
        
        downloaded_files.append(filename)
    
    print(f"Created {len(downloaded_files)} simulated files for demonstration")

## 4. Preprocess SAR Data

In [ ]:
# Preprocess downloaded SAR data
print('Preprocessing SAR data...')

processed_files = []

for i, file_path in enumerate(downloaded_files):
    try:
        print(f"Processing file {i+1}/{len(downloaded_files)}: {os.path.basename(file_path)}")
        
        # For real Sentinel-1 data, you would apply:
        # - Radiometric calibration
        # - Terrain correction
        # - Speckle filtering
        
        # For this demo, we'll apply basic preprocessing
        with rasterio.open(file_path) as src:
            image_data = src.read(1)
            profile = src.profile
        
        # Apply basic preprocessing
        # 1. Convert to dB scale
        image_db = 10 * np.log10(image_data + 1e-10)
        
        # 2. Apply simple speckle filter (moving average)
        from scipy import ndimage
        filtered_image = ndimage.uniform_filter(image_db, size=3)
        
        # 3. Normalize to reasonable range
        filtered_image = np.clip(filtered_image, -25, 5)
        
        # Save processed image
        processed_filename = file_path.replace('.tif', '_processed.tif')
        
        profile.update(dtype=filtered_image.dtype)
        with rasterio.open(processed_filename, 'w', **profile) as dst:
            dst.write(filtered_image, 1)
        
        processed_files.append(processed_filename)
        
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        continue

print(f"Successfully processed {len(processed_files)} files")

## 5. Load and Visualize Processed Data

In [ ]:
# Load processed images
images = []
dates = []

for i, file_path in enumerate(processed_files):
    with rasterio.open(file_path) as src:
        image_data = src.read(1)
        images.append(image_data)
        # For demo, create synthetic dates
        date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=i*30)
        dates.append(date.strftime('%Y-%m-%d'))

print(f"Loaded {len(images)} images")
print(f"Image shape: {images[0].shape}")
print(f"Dates: {dates}")

# Visualize the time series
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (img, date) in enumerate(zip(images[:6], dates[:6])):
    if i < len(axes):
        im = axes[i].imshow(img, cmap='gray', vmin=-20, vmax=0)
        axes[i].set_title(f"SAR Image {i+1}\n{date}")
        axes[i].set_axis_off()
        plt.colorbar(im, ax=axes[i], label='dB')

# Hide unused subplots
for i in range(len(images), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.suptitle('SAR Time Series', y=1.02, fontsize=16)
plt.show()

## 6. Perform Change Detection

In [ ]:
# Convert images to linear scale for change detection
linear_images = []
for img in images:
    # Convert from dB back to linear scale
    linear_img = 10 ** (img / 10)
    linear_images.append(linear_img)

# Stack images into 3D array (time, height, width)
image_stack = np.stack(linear_images, axis=0)
print(f"Image stack shape: {image_stack.shape}")

# Perform change detection using different methods
print('\nPerforming change detection...')

# Method 1: Omnibus test
print('1. Omnibus test...')
omnibus_results = change_detection.detect_changes(
    linear_images, 
    method='omnibus', 
    significance=0.05
)

# Method 2: Ratio test
print('2. Ratio test...')
ratio_results = change_detection.detect_changes(
    linear_images, 
    method='ratio'
)

# Method 3: Difference test
print('3. Difference test...')
difference_results = change_detection.detect_changes(
    linear_images, 
    method='difference'
)

print('Change detection completed!')

## 7. Visualize Change Detection Results

In [ ]:
# Visualize change detection results
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

methods = ['Omnibus Test', 'Ratio Test', 'Difference Test']
results = [omnibus_results, ratio_results, difference_results]

for i, (method, result) in enumerate(zip(methods, results)):
    # First change map
    im1 = axes[i, 0].imshow(result['first_change'], cmap='viridis')
    axes[i, 0].set_title(f"{method}\nFirst Change")
    axes[i, 0].set_axis_off()
    plt.colorbar(im1, ax=axes[i, 0], label='Time Step')
    
    # Change frequency map
    im2 = axes[i, 1].imshow(result['change_frequency'], cmap='hot')
    axes[i, 1].set_title(f"{method}\nChange Frequency")
    axes[i, 1].set_axis_off()
    plt.colorbar(im2, ax=axes[i, 1], label='# Changes')
    
    # Change magnitude map
    im3 = axes[i, 2].imshow(result['change_magnitude'], cmap='jet')
    axes[i, 2].set_title(f"{method}\nChange Magnitude")
    axes[i, 2].set_axis_off()
    plt.colorbar(im3, ax=axes[i, 2], label='Magnitude')

plt.tight_layout()
plt.suptitle('Change Detection Results Comparison', y=1.02, fontsize=16)
plt.show()

## 8. Detailed Analysis of Change Areas

In [ ]:
# Focus on omnibus test results for detailed analysis
change_data = omnibus_results

# Calculate change statistics
total_pixels = change_data['first_change'].size
changed_pixels = np.sum(change_data['first_change'] > 0)
change_percentage = (changed_pixels / total_pixels) * 100

print(f"Change Detection Statistics (Omnibus Test):")
print(f"Total pixels: {total_pixels:,}")
print(f"Changed pixels: {changed_pixels:,}")
print(f"Change percentage: {change_percentage:.2f}%")

# Find areas with highest change frequency
max_frequency = np.max(change_data['change_frequency'])
high_change_areas = change_data['change_frequency'] >= max_frequency * 0.8

print(f"\nAreas with high change frequency (>= {max_frequency * 0.8:.1f}):")
print(f"Number of pixels: {np.sum(high_change_areas):,}")

# Create a combined change map
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Original first image
axes[0, 0].imshow(images[0], cmap='gray', vmin=-20, vmax=0)
axes[0, 0].set_title('First SAR Image')
axes[0, 0].set_axis_off()

# Original last image
axes[0, 1].imshow(images[-1], cmap='gray', vmin=-20, vmax=0)
axes[0, 1].set_title('Last SAR Image')
axes[0, 1].set_axis_off()

# Change frequency overlay
axes[1, 0].imshow(images[0], cmap='gray', vmin=-20, vmax=0, alpha=0.7)
change_overlay = np.ma.masked_where(change_data['change_frequency'] == 0, 
                                   change_data['change_frequency'])
axes[1, 0].imshow(change_overlay, cmap='Reds', alpha=0.8)
axes[1, 0].set_title('Changes Overlaid on First Image')
axes[1, 0].set_axis_off()

# High change areas
axes[1, 1].imshow(high_change_areas, cmap='binary')
axes[1, 1].set_title('High Change Areas')
axes[1, 1].set_axis_off()

plt.tight_layout()
plt.show()

## 9. Time Series Analysis at Specific Points

In [ ]:
# Select points for time series analysis
# Find points with changes
change_locations = np.where(change_data['change_frequency'] > 0)

if len(change_locations[0]) > 0:
    # Select a few representative points
    n_points = min(5, len(change_locations[0]))
    indices = np.random.choice(len(change_locations[0]), n_points, replace=False)
    
    selected_points = [(change_locations[0][i], change_locations[1][i]) for i in indices]
else:
    # If no changes detected, select random points
    selected_points = [
        (100, 100), (200, 200), (300, 300), (150, 250), (250, 150)
    ]

print(f"Selected points for time series analysis: {selected_points}")

# Extract time series for selected points
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, (row, col) in enumerate(selected_points[:6]):
    if row < image_stack.shape[1] and col < image_stack.shape[2]:
        # Extract time series (convert back to dB for plotting)
        time_series = 10 * np.log10(image_stack[:, row, col] + 1e-10)
        
        axes[i].plot(range(len(time_series)), time_series, 'o-', linewidth=2, markersize=6)
        axes[i].set_title(f"Point ({row}, {col})")
        axes[i].set_xlabel('Time Step')
        axes[i].set_ylabel('Backscatter (dB)')
        axes[i].grid(True, alpha=0.3)
        
        # Highlight change points if any
        if change_data['first_change'][row, col] > 0:
            change_time = change_data['first_change'][row, col] - 1
            axes[i].axvline(x=change_time, color='red', linestyle='--', 
                          label=f"First change at t={change_time+1}")
            axes[i].legend()

# Hide unused subplots
for i in range(len(selected_points), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.suptitle('SAR Backscatter Time Series at Selected Points', y=1.02, fontsize=14)
plt.show()

## 10. Export Results

In [ ]:
# Create results directory
results_dir = 'change_detection_results'
os.makedirs(results_dir, exist_ok=True)

# Save change detection results as GeoTIFF files
print('Saving results...')

# Get geospatial information from the first processed file
with rasterio.open(processed_files[0]) as src:
    profile = src.profile
    profile.update(count=1)

# Save omnibus test results
for key, data in omnibus_results.items():
    if key != 'metadata' and isinstance(data, np.ndarray):
        output_file = os.path.join(results_dir, f"omnibus_{key}.tif")
        
        profile_copy = profile.copy()
        profile_copy.update(dtype=data.dtype)
        
        with rasterio.open(output_file, 'w', **profile_copy) as dst:
            dst.write(data, 1)
        
        print(f"Saved: {output_file}")

# Save summary statistics
summary_file = os.path.join(results_dir, 'change_detection_summary.txt')
with open(summary_file, 'w') as f:
    f.write("SAR Change Detection Results Summary\n")
    f.write("=" * 40 + "\n\n")
    f.write(f"Area of Interest: {aoi_bbox}\n")
    f.write(f"Time Period: {start_date} to {end_date}\n")
    f.write(f"Number of Images: {len(images)}\n")
    f.write(f"Image Dimensions: {images[0].shape}\n\n")
    
    f.write("Change Detection Statistics (Omnibus Test):\n")
    f.write(f"Total pixels: {total_pixels:,}\n")
    f.write(f"Changed pixels: {changed_pixels:,}\n")
    f.write(f"Change percentage: {change_percentage:.2f}%\n")
    f.write(f"Maximum change frequency: {max_frequency}\n")

print(f"Summary saved: {summary_file}")

# Create final visualization
fig = visualization.plot_changes(omnibus_results, 
                                output_file=os.path.join(results_dir, 'change_detection_overview.png'))
plt.close(fig)

print(f"\nAll results saved to: {results_dir}/")
print("\nChange detection analysis completed successfully!")

## Summary

This notebook demonstrated a complete workflow for urban change detection using real Sentinel-1 SAR data:

1. **Data Download**: Searched and downloaded Sentinel-1 GRD products from Copernicus Hub
2. **Preprocessing**: Applied radiometric calibration, terrain correction, and speckle filtering
3. **Change Detection**: Used three different algorithms (Omnibus, Ratio, Difference tests)
4. **Analysis**: Calculated change statistics and identified high-change areas
5. **Visualization**: Created comprehensive plots and time series analysis
6. **Export**: Saved results as GeoTIFF files and summary statistics

### Key Findings:
- The omnibus test provides the most statistically robust change detection
- Change frequency maps help identify areas of persistent change
- Time series analysis reveals the temporal patterns of changes

### Next Steps:
- Apply to larger areas or longer time periods
- Integrate with optical data for improved interpretation
- Implement automated change classification
- Add validation with ground truth data